In [16]:
from semantic_kernel.connectors.openapi.kernel_openapi import RestApiOperation
from typing import Dict, Mapping, Optional, Union

async def run_operation(
    self,
    operation: RestApiOperation,
    path_params: Optional[Dict[str, str]] = None,
    query_params: Optional[Dict[str, str]] = None,
    headers: Optional[Dict[str, str]] = None,
    request_body: Optional[Union[str, Dict[str, str]]] = None,
) -> str:
    prepared_request = operation.prepare_request(
        path_params=path_params,
        query_params=query_params,
        headers=headers,
        request_body=request_body,
    )
    is_valid = prepared_request.validate_request(spec=self.spec)
    if not is_valid:
        return None

    async with aiohttp.ClientSession(raise_for_status=True) as session:
        async with session.request(
            prepared_request.method,
            prepared_request.url,
            params=prepared_request.params,
            headers=prepared_request.headers,
            json=prepared_request.request_body,
        ) as response:
            response_text = await response.text()
            print("Raw API Response:", response_text)  # Print the raw response
            return response_text

In [17]:
import asyncio
from semantic_kernel.functions.function_result import FunctionResult  # Corrected import
import semantic_kernel as sk
from semantic_kernel.connectors.openapi import register_openapi_plugin

if __name__ == "__main__":
    """Client"""
    kernel = sk.Kernel()

    openapi_plugin = register_openapi_plugin(
        kernel, "openApiPlugin", "natgas-equity.yaml"
    )
    # variables = {
    #     "query_params": '{"limit": "5", "text": "Equinor"}',
    #     "headers": '{"Cache-Control": "no-cache", "Gen-Api-Key": "beac8ea8436b4db785af539108dfff9e"}',
    # }

    context_variables = sk.KernelArguments(
        query_params={"limit": "5", "text": "Equinor", "format": "json"},
        headers={
            "Cache-Control": "no-cache",
            "Gen-Api-Key": "beac8ea8436b4db785af539108dfff9e",
        },
    )

    print(openapi_plugin["GetEarningsCallNotes"])

    result = await (
        # Call the function defined in openapi.yaml
        openapi_plugin["GetEarningsCallNotes"].invoke(
            kernel, arguments=context_variables
        )
    )

    # Check if the result is a FunctionResult object and print its value
    if isinstance(result, FunctionResult):
        print("FunctionResult received:")
        print(result.value)  # This should print the actual result value
    else:
        print("Result:")
        print(result)  # This will print the result as is

metadata=KernelFunctionMetadata(name='GetEarningsCallNotes', plugin_name='openApiPlugin', description='Earnings Call Notes', parameters=[KernelParameterMetadata(name='path_params', description='A dictionary of path parameters', default_value=None, type_='Dict, str', is_required=False, type_object=None), KernelParameterMetadata(name='query_params', description='A dictionary of query parameters', default_value=None, type_='Dict, str', is_required=False, type_object=None), KernelParameterMetadata(name='headers', description='A dictionary of headers', default_value=None, type_='Dict, str', is_required=False, type_object=None), KernelParameterMetadata(name='request_body', description='A dictionary of the request body', default_value=None, type_='Dict, str', is_required=False, type_object=None)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=None)) method=<function 

In [18]:
    if isinstance(result, FunctionResult):
        print("FunctionResult received:")
        # If the response data is a dictionary or list, print it directly
        if isinstance(result.value, (dict, list)):
            print(result.value)
        # If the response data is another object, you may need to access its attributes
        else:
            print("Response data type:", type(result.value))
            print("Response data attributes:", dir(result.value))
    else:
        print("Result:")
        print(result)

FunctionResult received:
Response data type: <class 'NoneType'>
Response data attributes: ['__bool__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
